# Feature Engineering
Para consultar nuestra API, requerimos una nueva columna que indique los sentimientos de las reseñas de usuarios. Usaremos una reconocida librería de procesamiento de lenguaje natural para facilitar este proceso.

## Importaciones

In [106]:
import pandas as pd
import json
from textblob import TextBlob

## Preparación de conjuntos de datos

In [107]:
# Extraer datos desde los archivos 'parquet' con información que se preparo en la etapa de ETL
df_games = pd.read_parquet('../Datasets/steam_games.parquet')
df_items = pd.read_parquet('../Datasets/user_items.parquet')
df_reviews = pd.read_parquet('../Datasets/user_reviews.parquet')

In [108]:
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'tags',
       'reviews_url', 'specs', 'price', 'early_access', 'item_id', 'developer',
       'release_year'],
      dtype='object')

In [109]:
df_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'steam_id', 'items_count', 'user_id', 'user_url'],
      dtype='object')

In [110]:
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'helpful', 'recommend',
       'review'],
      dtype='object')

## Análisis de sentimientos
Se pide crear una nueva columna llamada 'sentiment_analysis' que reemplace a 'review' donde se realice un análisis de sentimiento de los comentarios con la siguiente escala:

0 si es malo,<br>
1 si es neutral o esta sin review<br>
2 si es positivo.

In [111]:
from textblob import TextBlob

# Función para clasificar el sentimiento según la escala dada
def classify_sentiment(polarity):
    if polarity < -0.2:
        return 0
    elif polarity > 0.2:
        return 2
    else:
        return 1

# Aplicar el análisis de sentimiento y crear la nueva columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: classify_sentiment(TextBlob(x).sentiment.polarity))

Finalmente, se elimina la columna de 'review'.

In [112]:
df_reviews = df_reviews.drop(columns=['review'])
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'helpful', 'recommend',
       'sentiment_analysis'],
      dtype='object')

In [97]:
#Verificamos si hay nulos
df_reviews.isnull().sum().sort_values(ascending=False)

user_id               0
posted                0
item_id               0
recommend             0
sentiment_analysis    0
dtype: int64

## Género y el año con más horas jugadas

Se crea el dataframe 'max_playtime_by_genre_year' con la información del año con más horas jugadas por género.

In [98]:
# Crear un dataframe combinando la información necesaria
combined_df = pd.merge(df_items, df_games[['item_id', 'release_year', 'genres']], on='item_id', how='left')

# Agrupar por género, año y sumar las horas jugadas
grouped = combined_df.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()

# Encontrar el año con más horas jugadas por género
max_playtime_by_genre_year = grouped.loc[grouped.groupby('genres')['playtime_forever'].idxmax()]

# Convertir la columna 'release_year' de flotante a entero
max_playtime_by_genre_year['release_year'] = max_playtime_by_genre_year['release_year'].astype(int)

# Mostrar el resultado
max_playtime_by_genre_year



,genres,release_year,playtime_forever
26,Action,2012,1085635110
59,Adventure,2011,221707756
71,Animation & Modeling,2015,1345545
75,Audio Production,2014,455463
104,Casual,2015,81708365
109,Design & Illustration,2012,1930339
116,Early Access,2013,118902893
124,Education,2013,340567
141,Free to Play,2013,146213734
158,Indie,2006,446115272


Dataframe con el usuario que acumula más horas jugadas por género

In [99]:
# Combinar los dataframes df_games y df_items en función de 'item_id'
merged_df = pd.merge(df_games, df_items, on='item_id')

# Convertir las horas jugadas a horas en lugar de minutos
#merged_df['playtime_hours'] = merged_df['playtime_forever'] / 60

# Calcular la suma total de horas jugadas por usuario y género
user_genre_playtime = merged_df.groupby(['user_id', 'genres'])['playtime_forever'].sum().reset_index()

# Encontrar el usuario que acumula más horas jugadas por género
max_playtime_per_genre = user_genre_playtime.loc[user_genre_playtime.groupby('genres')['playtime_forever'].idxmax()]


# Imprimir o trabajar con los resultados
print("Dataframe con el usuario que acumula más horas jugadas por género:")
print(max_playtime_per_genre)

Dataframe con el usuario que acumula más horas jugadas por género:
                  user_id                 genres  playtime_forever
485519            Sp3ctre                 Action           1692650
472192       REBAS_AS_F-T              Adventure           2182066
479391         ScottyG555   Animation & Modeling            168314
447836        Lickidactyl       Audio Production            109916
472195       REBAS_AS_F-T                 Casual           1224933
479393         ScottyG555  Design & Illustration            168314
18051   76561197978756659           Early Access            316969
160609  76561198059330972              Education             65427
570659         idonothack           Free to Play            808241
472200       REBAS_AS_F-T                  Indie           2401378
417826        Evilutional  Massively Multiplayer            688260
483274           Sky_Wolf          Photo Editing             21885
634706        shinomegami                    RPG           102

In [100]:
max_playtime_per_genre.columns

Index(['user_id', 'genres', 'playtime_forever'], dtype='object')

Dataframe con la acumulación de horas jugadas por año de los usuarios del dataframe 'max_playtime_per_genre'

In [101]:
# Obtener los usuarios contenidos en el dataframe 'max_playtime_per_genre'
users_of_interest = max_playtime_per_genre['user_id'].unique()

# Filtrar el dataframe original de items para incluir solo a estos usuarios
filtered_items_df = df_items[df_items['user_id'].isin(users_of_interest)]

# Combinar los dataframes df_games y el filtrado de items en función de 'item_id'
filtered_merged_df = pd.merge(df_games, filtered_items_df, on='item_id')

# Calcular la suma total de horas jugadas por año
playtime_per_year = filtered_merged_df.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()
playtime_per_year.rename(columns={'playtime_forever': 'playtime_total_minutes'}, inplace=True)

# Convertir las horas jugadas a horas en lugar de minutos
playtime_per_year['playtime_total_hours'] = playtime_per_year['playtime_total_minutes'] / 60

# Resumen de horas jugadas por año de los usuarios seleccionados por género
summary_playtime_per_year = playtime_per_year.groupby(['user_id', 'release_year'])['playtime_total_hours'].sum().reset_index()

print(summary_playtime_per_year)

               user_id  release_year  playtime_total_hours
0    76561197978756659        1998.0              0.000000
1    76561197978756659        1999.0              0.133333
2    76561197978756659        2000.0              0.000000
3    76561197978756659        2001.0              0.000000
4    76561197978756659        2003.0              0.000000
..                 ...           ...                   ...
292        shinomegami        2013.0          18116.283333
293        shinomegami        2014.0          18525.700000
294        shinomegami        2015.0          18367.966667
295        shinomegami        2016.0           2246.016667
296        shinomegami        2017.0              6.750000

[297 rows x 3 columns]


## Dataframe con los top 3 juegos mas recomendado por año

In [102]:
# Filtrar las reseñas que cumplen con las condiciones
filtered_reviews = df_reviews[(df_reviews['recommend'] == True) & (df_reviews['sentiment_analysis'].isin([1,2]))].copy()

# Extraer el año de la columna 'posted'
filtered_reviews['year'] = pd.to_datetime(filtered_reviews['posted']).dt.year

# Combinar los DataFrames de juegos y reseñas
merged_df = pd.merge(filtered_reviews, df_games, on='item_id')

# Agrupar por año y juego, contar las recomendaciones y obtener los top 3 juegos por año
top3_recomendados = (merged_df.groupby(['year', 'title'])
                     .agg(recomendaciones=('recommend', 'count'))
                     .reset_index()
                     .sort_values(by=['year', 'recomendaciones'], ascending=[True, False])
                     .groupby('year')
                     .head(3))

# Ahora top3_recomendados contiene el top 3 de juegos más recomendados por año
top3_recomendados


,year,title,recomendaciones
27,2010,Team Fortress 2,20
10,2010,Garry's Mod,6
11,2010,Killing Floor,6
164,2011,Team Fortress 2,140
165,2011,Terraria,76
134,2011,Portal 2,50
364,2012,Team Fortress 2,480
366,2012,Terraria,148
324,2012,Realm of the Mad God,75
869,2013,Team Fortress 2,1472


## Dataframe con los top 3 juegos no recomendados por año

In [103]:
# Filtrar los juegos menos recomendados y comentarios negativos
negative_reviews = df_reviews[(df_reviews['recommend'] == False) & (df_reviews['sentiment_analysis'] == 0)].copy()

# Extraer el año de la columna 'posted'
negative_reviews['year'] = pd.to_datetime(negative_reviews['posted']).dt.year

# Combinar los DataFrames utilizando la columna 'item_id'
merged_data = pd.merge(df_games, negative_reviews, on='item_id')

# Obtener el top 3 de juegos menos recomendados por año
top3_no_recomendados = merged_data.groupby(['year', 'title']).size().reset_index(name='count')
top3_no_recomendados = top3_no_recomendados.sort_values(['year', 'count']).groupby('year').head(3)

top3_no_recomendados


,year,title,count
1,2011,Men of War: Vietnam,1
0,2011,And Yet It Moves,2
2,2011,Team Fortress 2,2
3,2012,Red Faction®: Armageddon™,1
4,2012,Team Fortress 2,2
12,2013,Commander Keen,1
22,2013,Injustice: Gods Among Us Ultimate Edition,1
23,2013,Iron Warriors: T - 72 Tank Command,1
63,2014,Age of Empires II HD,1
66,2014,Alpha Prime,1


## Dataframe con el conteo de registros para cada sentimiento y por año de lanzamiento

In [104]:
# Fusionar los dataframes en base a la columna 'item_id'
merged_df = pd.merge(df_games, df_reviews, on='item_id')

# Filtrar las filas donde el análisis de sentimiento no es nulo
filtered_df = merged_df[merged_df['sentiment_analysis'].notnull()]

# Mapear los valores de 'sentiment_analysis' a los tipos de sentimiento
sentiment_mapping = {0: 'Negativo', 1: 'Neutral', 2: 'Positivo'}
filtered_df['sentiment_label'] = filtered_df['sentiment_analysis'].map(sentiment_mapping)

# Agrupar por 'release_year' y 'sentiment_label', contar registros y reindexar para tener una estructura de dataframe adecuada
result = filtered_df.groupby(['release_year', 'sentiment_label']).size().reset_index(name='count')

# Reestructurar los datos para tener un DataFrame con los tres tipos de sentimientos como columnas y el año como columna independiente
conteo_sentimientos = result.pivot(index='release_year', columns='sentiment_label', values='count').fillna(0).reset_index()

# Mostrar el resultado
print(conteo_sentimientos)


sentiment_label  release_year  Negativo  Neutral  Positivo
0                      1989.0       0.0      2.0       0.0
1                      1990.0       1.0      0.0       4.0
2                      1991.0       0.0      0.0       1.0
3                      1992.0       0.0      4.0       2.0
4                      1993.0       0.0      6.0       0.0
5                      1994.0       0.0      6.0       5.0
6                      1995.0       1.0      9.0       4.0
7                      1996.0       1.0      0.0       4.0
8                      1997.0       1.0     31.0      16.0
9                      1998.0       6.0     67.0      19.0
10                     1999.0       6.0     37.0       8.0
11                     2000.0       6.0     54.0      28.0
12                     2001.0       1.0     24.0       6.0
13                     2002.0       1.0     37.0      10.0
14                     2003.0      67.0    289.0      77.0
15                     2004.0      31.0    287.0     180

## Carga de los dataframe
Se almacenan los dataframes en formato Parquet para optimizar la estructura de datos en el despliegue.

In [105]:
dfs = [df_reviews,max_playtime_by_genre_year, max_playtime_per_genre, summary_playtime_per_year,top3_recomendados,top3_no_recomendados,conteo_sentimientos]
# Nombres correspondientes a cada DataFrame
names = ['user_reviews','max_playtime_by_genre_year', 'max_playtime_per_genre', 'summary_playtime_per_year','top3_recomendados','top3_no_recomendados','conteo_sentimientos']

for df, name in zip(dfs, names):
    archivo = f'../Datasets/{name}.parquet'
    df.to_parquet(archivo)
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame 'max_playtime_by_genre_year' guardado como '../Datasets/max_playtime_by_genre_year.parquet'
DataFrame 'max_playtime_per_genre' guardado como '../Datasets/max_playtime_per_genre.parquet'
DataFrame 'summary_playtime_per_year' guardado como '../Datasets/summary_playtime_per_year.parquet'
DataFrame 'top3_recomendados' guardado como '../Datasets/top3_recomendados.parquet'
DataFrame 'top3_no_recomendados' guardado como '../Datasets/top3_no_recomendados.parquet'
DataFrame 'conteo_sentimientos' guardado como '../Datasets/conteo_sentimientos.parquet'
